In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score

In [2]:
df = pd.read_csv('train.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df['Soil Type'].value_counts().plot(kind='bar')

In [ ]:
df['Crop Type'].value_counts().plot(kind='bar')

In [ ]:
df['Fertilizer Name'].value_counts().plot(kind='bar')

In [ ]:
df_fert = pd.get_dummies(df,columns=['Fertilizer Name'])
df_fert.groupby(['Soil Type','Crop Type'])[df_fert.columns[9:]].sum().style.background_gradient('viridis')

In [ ]:
df = pd.get_dummies(df, columns=['Soil Type','Crop Type'])
df.head()

In [ ]:
df['N vs K'] = df['Nitrogen'] > df['Potassium']
df['N vs P'] = df['Nitrogen'] > df['Phosphorous']
df['K vs P'] = df['Potassium'] > df['Phosphorous']

In [ ]:
X = df.drop(['Fertilizer Name','id'],axis=1)
y = df['Fertilizer Name']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [ ]:
rfc = RandomForestClassifier(n_estimators=20,max_depth=10)
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
def mapk(actual, predicted, k=3):
    def apk(a, p, k):
        p = p[:k]
        score = 0.0
        hits = 0.0
        seen = set()
        for i, pred in enumerate(p):
            if pred in a and pred not in seen:
                hits += 1.0
                score += hits / (i + 1.0)
                seen.add(pred)
        return score / min(len(a), k)

    actual_wrapped = [[a] for a in actual]
    return np.mean([apk(a, p, k) for a, p in zip(actual_wrapped, predicted)])

In [ ]:
classdict={i:n for i,n in enumerate(rfc.classes_)}

In [ ]:
df_test = pd.read_csv('test.csv')
df_test = df = pd.get_dummies(df_test, columns=['Soil Type','Crop Type'])
df_test['N vs K'] = df_test['Nitrogen'] > df_test['Potassium']
df_test['N vs P'] = df_test['Nitrogen'] > df_test['Phosphorous']
df_test['K vs P'] = df_test['Potassium'] > df_test['Phosphorous']

In [ ]:
X_test = df.drop(['id'],axis=1)
y_pred_probs = rfc.predict_proba(X_test)

In [ ]:
pred_list=[]
for i in np.argsort(-y_pred_probs)[:,:3]:
    pred_list.append(classdict[i[0]]+' '+classdict[i[1]]+' '+classdict[i[2]])
pred_list=pd.DataFrame(data=pred_list,columns=['Fertilizer Name'])
df_sub = pd.concat([df_test['id'],pred_list],axis=1)
df_sub.to_csv('submission.csv',index=False)

In [ ]:
df_sub